# Obtain the SVR Model Using the Fullset

In [ ]:
# Scikit-Learn
from sklearn.preprocessing import quantile_transform
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import svm

# Matplotlib 
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

# Data Structure and Math
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import math

# Descriptor Calculators
import padelpy
from padelpy import padeldescriptor
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from mordred import Calculator, descriptors

# Model IO
from joblib import dump, load

In [ ]:
# define RMSE
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# define the SVR model
svr = svm.SVR(gamma='scale',C=20,epsilon=0.01)

# define a function to get model's output efficiently
def ml_model(splited_data,regressor_object):
    X_train, X_test, y_train, y_test = \
        splited_data[0], splited_data[1], splited_data[2], splited_data[3]
    regressor_object.fit(X_train, y_train)
    # get predictions from the train set
    train_prediction = regressor_object.predict(X_train)
    R2_train = np.square(pearsonr(y_train, train_prediction)[0])
    RMSE_train = rmse(y_train, train_prediction)
    # get the prediction from the test set
    test_prediction = regressor_object.predict(X_test)
    R2_test = np.square(pearsonr(y_test, test_prediction)[0])
    RMSE_test = rmse(y_test, test_prediction)
    # generate formatted output in the list
    return [R2_train, RMSE_train, R2_test, RMSE_test, test_prediction], regressor_object

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/REMUU/IonEner-Pred/main/datasets/full_set/nist_organic_full_set.csv')

In [ ]:
# get name of descriptors
des_cols = data.iloc[:,5:].columns

# scale the data by quantile_transform
X = quantile_transform(data.iloc[:,5:])
y = data['IE']
kf = KFold(n_splits=10)
kf.get_n_splits(X)

# store metrics during each fold
test_cv_df = pd.DataFrame(columns=['Regressor', 'R^2', 'RMSE', 'Fold Number'])

# run the 10-Fold-CV for SVR
y_predicted_list = []
y_experimental_list = []


In [ ]:
# loop for CVs of sklearn and xgb objects
fold_number = 0
for train_index, test_index in kf.split(X):
    fold_number += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    splited_data = [X_train, X_test, y_train, y_test]

    metrics, mo = ml_model(splited_data, svr)

    test_new_row = pd.DataFrame([['SVR', metrics[2], metrics[3], fold_number]], columns= \
                                ['Regressor', 'R^2', 'RMSE', 'Fold Number'])
    test_cv_df = test_cv_df.append(test_new_row, ignore_index=True)

    y_predicted_list.extend(metrics[4])
    y_experimental_list.extend(y_test)
    print('SVR', 'at fold ', fold_number)
    print('RMSE: {} eV; R2: {}.'.format(round(metrics[3],3), round(metrics[2],3)))
    
    dump(mo, '{}_fold_svr.joblib'.format(fold_number))

    for i in range(len(test_index)):
        ind_name = test_index[i]
        mol = data.iloc[:,:5].iloc[[ind_name]].values[0]
        new_row = pd.DataFrame([[mol[0], mol[1], mol[2], mol[3], mol[4], metrics[4][i]]], \
            columns=['CAS Name','InChI','CAS Link','smiles','IE', 'Predicted IE'])


In [ ]:
print('RMSE and R^2 during the 10-Fold-CV is {} eV and {}.'.format(round(np.mean(test_cv_df['RMSE']),3),round(np.mean(test_cv_df['R^2']),3)))
print('Please check the metrics. If it defer from values in the paper, something must be going wrong and the result should not be used or compared.')
print('Notice: The RMSE and R^2 in the paper is 0.405 eV and 0.865.')

# Predict IE of the New Molecule Here

### Please Enter Your SMILES in the Next Code Block

In [ ]:
# input your molecules in ['smiles', boolean of radical] format and in the [] for storing multiple molecules
# if it is radical, input 1 and if not input 0
new_records = [['CCOC=O',0],['CCC1C2CC3CC(C2)CC1C3',0]] 

### Check elements out of application domain

In [ ]:
# the application domain is limited to following elements where elements outside of the range will let to expected higher errors
appDomain = ['H', 
             'B', 'C', 'N', 'O','F',
             'Si', 'P', 'S', 'Cl',
             'Ge', 'As', 'Se', 'Br',
             'I',]


atomInAppDomain = []
atomNotInAppDomain = []

for rec in new_records:
    smi = rec[0]
    mol = Chem.MolFromSmiles(smi)
    for atom in mol.GetAtoms():
        at = atom.GetSymbol()
        if at in appDomain and at not in atomInAppDomain:
            atomInAppDomain.append(at)
            continue
        if at not in appDomain and at not in atomNotInAppDomain:
            atomNotInAppDomain.append(at)

aInA = ''
for i in atomInAppDomain:
    if aInA == '':
        aInA = i
        continue
    aInA = aInA + ', ' + i

aNotInA = ''
for i in atomNotInAppDomain:
    if aNotInA == '':
        aNotInA = i
        continue
    aNotInA = aNotInA + ', ' + i


print('Elements {} are elements that we used to train those model. It is likely that the user will get a similar performance with our study.'.format(aInA))
if atomNotInAppDomain != []:
    print('!!!!!!!!!!!!!!!!!!!!!!WARNING!!!!!!!!!!!!!!!!!!!!!!')
    print("Following elements are not presented in the training of models. Our parameters may generate very different results as the user's expectation. Please Use with Cautions.")
    print(aNotInA)

In [ ]:
# calculate descriptors
# please note that this work flow is designed for fast implementation in the COLAB interface
# the result may be varied from the approach described in the paper
calc = Calculator(descriptors, ignore_3D=True)
padeldescriptor(descriptortypes='./descriptors.xml')
df_pred = pd.DataFrame()

for rec in new_records:
  smi = rec[0]
  rad = rec[1]
  try:
    mol = Chem.MolFromSmiles(smi)
  except:
    print('SMILES: {} is invalid, please check your input.')
  # calculate and merge descriptors
  mordred_descriptors = calc(mol)
  mordred_df = pd.DataFrame([float(i[1]) for i in mordred_descriptors.items()], index=[str(i[0]) for i in mordred_descriptors.items()]).T
  maccs_pubchem_descriptors = padelpy.from_smiles(smi, fingerprints=True, descriptors=False, output_csv=None)
  maccs_pubchem_df = pd.DataFrame([[float(i[1]) for i in maccs_pubchem_descriptors.items()]], columns=[i[0] for i in maccs_pubchem_descriptors.items()])
  descriptor_df = pd.concat([pd.DataFrame([[smi,rad]],columns=['SMILES','BinaryRadical']),mordred_df,maccs_pubchem_df],axis=1)
  df_pred = pd.concat([df_pred,descriptor_df], axis=0)


## Make Prediction Here

### Best of 10

In [ ]:
X_test = quantile_transform(df_pred[des_cols])

ie_preds = []
fold_counts = [i+1 for i in range(10)]

for f in fold_counts:
    reg = load('{}_fold_svr.joblib'.format(f))
    ie_preds.append(reg.predict(X_test))

print(ie_preds)
print(np.average(ie_preds,axis=0))
print(data[data['smiles']=='CCOC=O']['IE'])
print(data[data['smiles']=='CCC1C2CC3CC(C2)CC1C3']['IE'])



### Best of 3

In [ ]:
X_test = quantile_transform(df_pred[des_cols])

ie_preds = []
fold_counts = [1, 9, 10]

for f in fold_counts:
    reg = load('{}_fold_svr.joblib'.format(f))
    ie_preds.append(reg.predict(X_test))

print(ie_preds)
print(np.average(ie_preds,axis=0))
print(data[data['smiles']=='CCOC=O']['IE'])
print(data[data['smiles']=='CCC1C2CC3CC(C2)CC1C3']['IE'])

### Best of Best

In [ ]:
X_test = quantile_transform(df_pred[des_cols])

ie_preds = []
fold_counts = [1]

for f in fold_counts:
    reg = load('{}_fold_svr.joblib'.format(f))
    ie_preds.append(reg.predict(X_test))

print(ie_preds)
print(np.average(ie_preds,axis=0))
print(data[data['smiles']=='CCOC=O']['IE'])
print(data[data['smiles']=='CCC1C2CC3CC(C2)CC1C3']['IE'])